In [23]:
import pandas as pd
import numpy as np
import inspect
import math
import joblib
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

cv_splitter = ShuffleSplit(n_splits=5, test_size=0.25, random_state=42)
csvcompleto = pd.read_csv("data.csv", sep=',')
csvcompleto


,track0,track2,track4,track6,track8,track10,track12,track14,track16,track18,speedX,angleToTrackAxis,trackPosition,distanceFromStartLine,steering,acceleration,brake
0,7.33374,7.82722,9.62860,14.7754,39.8054,21.1142,7.24756,4.75464,3.88989,3.66663,0.000304,0.008929,-0.333363,5759.10,0.000000,0.0,0.0
1,7.33374,7.82722,9.62860,14.7754,39.8054,21.1142,7.24756,4.75464,3.88989,3.66663,0.000304,0.008929,-0.333363,5759.10,0.000000,0.0,0.0
2,7.33374,7.82722,9.62860,14.7754,39.8054,21.1142,7.24756,4.75464,3.88989,3.66663,0.000304,0.008929,-0.333363,5759.10,0.000000,0.0,0.0
3,7.33374,7.82722,9.62860,14.7754,39.8054,21.1142,7.24756,4.75464,3.88989,3.66663,0.000304,0.008929,-0.333363,5759.10,0.000000,0.0,0.0
4,7.33374,7.82722,9.62860,14.7754,39.8054,21.1142,7.24756,4.75464,3.88989,3.66663,0.000304,0.008929,-0.333363,5759.10,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55302,6.69963,7.00605,8.40409,12.3619,30.2656,34.2499,9.41677,5.86919,4.67249,4.31328,172.804000,-0.048411,-0.216687,4099.32,0.005775,1.0,0.0
55303,6.65152,6.95401,8.33907,12.2595,29.9558,34.8238,9.53512,5.93876,4.72622,4.36176,173.022000,-0.049103,-0.207909,4100.37,-0.007785,1.0,0.0
55304,6.60242,6.90099,8.27294,12.1557,29.6445,35.4097,9.65581,6.00972,4.78104,4.41123,173.248000,-0.049787,-0.198952,4101.43,-0.011401,1.0,0.0
55305,6.55222,6.84691,8.20571,12.0507,29.3340,36.0023,9.77864,6.08210,4.83702,4.46179,173.474000,-0.050442,-0.189797,4102.49,-0.005375,1.0,0.0


In [24]:
features = csvcompleto.iloc[:,:14]
features

,track0,track2,track4,track6,track8,track10,track12,track14,track16,track18,speedX,angleToTrackAxis,trackPosition,distanceFromStartLine
0,7.33374,7.82722,9.62860,14.7754,39.8054,21.1142,7.24756,4.75464,3.88989,3.66663,0.000304,0.008929,-0.333363,5759.10
1,7.33374,7.82722,9.62860,14.7754,39.8054,21.1142,7.24756,4.75464,3.88989,3.66663,0.000304,0.008929,-0.333363,5759.10
2,7.33374,7.82722,9.62860,14.7754,39.8054,21.1142,7.24756,4.75464,3.88989,3.66663,0.000304,0.008929,-0.333363,5759.10
3,7.33374,7.82722,9.62860,14.7754,39.8054,21.1142,7.24756,4.75464,3.88989,3.66663,0.000304,0.008929,-0.333363,5759.10
4,7.33374,7.82722,9.62860,14.7754,39.8054,21.1142,7.24756,4.75464,3.88989,3.66663,0.000304,0.008929,-0.333363,5759.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55302,6.69963,7.00605,8.40409,12.3619,30.2656,34.2499,9.41677,5.86919,4.67249,4.31328,172.804000,-0.048411,-0.216687,4099.32
55303,6.65152,6.95401,8.33907,12.2595,29.9558,34.8238,9.53512,5.93876,4.72622,4.36176,173.022000,-0.049103,-0.207909,4100.37
55304,6.60242,6.90099,8.27294,12.1557,29.6445,35.4097,9.65581,6.00972,4.78104,4.41123,173.248000,-0.049787,-0.198952,4101.43
55305,6.55222,6.84691,8.20571,12.0507,29.3340,36.0023,9.77864,6.08210,4.83702,4.46179,173.474000,-0.050442,-0.189797,4102.49


In [25]:
labels = csvcompleto.iloc[:,-3:]
labels

,steering,acceleration,brake
0,0.000000,0.0,0.0
1,0.000000,0.0,0.0
2,0.000000,0.0,0.0
3,0.000000,0.0,0.0
4,0.000000,0.0,0.0
...,...,...,...
55302,0.005775,1.0,0.0
55303,-0.007785,1.0,0.0
55304,-0.011401,1.0,0.0
55305,-0.005375,1.0,0.0


In [26]:
X_temp, X_test, y_temp, y_test = train_test_split(
    features, labels,
    test_size=0.2,
    random_state=42
)

In [27]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', MLPRegressor(max_iter=1000, early_stopping=True, random_state=42))
])

'''paramGrid = {
    "mlp__hidden_layer_sizes": (10,8,4),
    "mlp__activation": ["relu", "tanh"],
    "mlp__solver": ["adam", "lbfgs"],
    "mlp__alpha": [0.0001, 0.001, 0.01],  # regolarizzazione L2
    "mlp__learning_rate": ["constant", "adaptive"]
}'''

paramGrid = {
    "mlp__hidden_layer_sizes": [(10, 8, 4)],
    "mlp__activation": ["relu"],
    "mlp__solver": ["adam"],
    "mlp__alpha": [0.001],
    "mlp__learning_rate": ["constant"]
}

gridsearch = GridSearchCV(estimator=pipeline,
                          param_grid=paramGrid,
                          scoring="neg_mean_squared_error",
                          cv=cv_splitter,
                          verbose=2,
                          n_jobs=-1)

print("Starting grid search...")
gridsearch.fit(X_temp, y_temp)

print("\nBest parameters:", gridsearch.best_params_)
print("Best CV score:", gridsearch.best_score_)

# Predizioni sul test set
best_model = gridsearch.best_estimator_
y_pred = best_model.predict(X_test)

# Metriche finali
test_mse = mean_squared_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

print(f"\nTest Set Performance:")
print(f"MSE: {test_mse:.6f}")
print(f"RMSE: {np.sqrt(test_mse):.6f}")
print(f"R²: {test_r2:.6f}")

# Analisi risultati dettagliata
results_df = pd.DataFrame(gridsearch.cv_results_)

# Migliori configurazioni
print("\nTop 5 configurazioni:")
top_results = results_df.nlargest(5, 'mean_test_score')[
    ['params', 'mean_test_score', 'std_test_score', 'rank_test_score']
]
for idx, row in top_results.iterrows():
    print(f"Rank {row['rank_test_score']}: {row['params']}")
    print(f"  Score: {row['mean_test_score']:.6f} (±{row['std_test_score']:.6f})")

# Salva il miglior modello
joblib.dump(best_model, 'best_mlp_model.pkl')
print("\nModello salvato come 'best_mlp_model.pkl'")

# Visualizza DataFrame dei risultati
print(f"\nGrid search completato. Testati {len(results_df)} combinazioni di parametri.")
results_df.head()

Starting grid search...
Fitting 5 folds for each of 1 candidates, totalling 5 fits

Best parameters: {'mlp__activation': 'relu', 'mlp__alpha': 0.001, 'mlp__hidden_layer_sizes': (10, 8, 4), 'mlp__learning_rate': 'constant', 'mlp__solver': 'adam'}
Best CV score: -0.0467833994703929

Test Set Performance:
MSE: 0.035197
RMSE: 0.187610
R²: 0.420115

Top 5 configurazioni:
Rank 1: {'mlp__activation': 'relu', 'mlp__alpha': 0.001, 'mlp__hidden_layer_sizes': (10, 8, 4), 'mlp__learning_rate': 'constant', 'mlp__solver': 'adam'}
  Score: -0.046783 (±0.001217)

Modello salvato come 'best_mlp_model.pkl'

Grid search completato. Testati 1 combinazioni di parametri.


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_mlp__activation,param_mlp__alpha,param_mlp__hidden_layer_sizes,param_mlp__learning_rate,param_mlp__solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,3.779,1.810399,0.0036,0.00049,relu,0.001,"(10, 8, 4)",constant,adam,"{'mlp__activation': 'relu', 'mlp__alpha': 0.00...",-0.045131,-0.047038,-0.048204,-0.04791,-0.045634,-0.046783,0.001217,1


<bound method Pipeline.predict of Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 MLPRegressor(alpha=0.001, early_stopping=True,
                              hidden_layer_sizes=(10, 8, 4), max_iter=1000,
                              random_state=42))])>